In [1]:
import pandas as pd
import numpy as np
import json

# Load User Transaction Data
Load user transaction history file from `json` obtained from the AAVE v2 subgraph.

In [20]:
with open("aave_user_transaction_data_complete.json") as file:
    response = json.load(file)

In [3]:
user_data = response

## Formating with format_tx
Function that formats transactions as obtained from The Graph so they can later be used to create a pandas DataFrame with all changes in user positions. 
This function returns a dictionary with the following keys:

- `tx_id` : Id of transaction on AAVE v2.


- `tx_type` : Type of transaction from `['deposit','redeem','borrow','repay','liquidation']`.


- `timestamp` : Timestamp at which the transaction was effectuated.


- `account` : Address of the user whose position was affected by the transaction.


- `collateral_<reserve>` : Change in collateral for <reserve> token, positive if depositing and negative if redeeming.

    
- `principal_<reserve>` : Change in loan amount for <reserve> token, positive if borrowing and negative if repaying.

In [4]:
def format_tx(tx, user_id):
    tx_row = {}
    
    
    tx_row['tx_id'] = tx['id']
    tx_row['timestamp'] = tx['timestamp']
    tx_row['account'] = user_id
    
    if tx_type in ['deposit','redeem']:
        
        sign = 1 if tx_type == 'deposit' else -1
        
        tx_row['collateral_'+tx['reserve']['symbol']] = sign * int(tx['amount'])*(10**(-tx['reserve']['decimals']))
        
        tx_row['principal_'+tx['reserve']['symbol']] = 0.0
        
    elif tx_type in ['borrow','repay']:
        
        sign = 1 if tx_type == 'borrow' else -1
        
        tx_row['collateral_'+tx['reserve']['symbol']] = 0.0
        
        tx_row['principal_'+tx['reserve']['symbol']] = sign * int(tx['amount'])*(10**(-tx['reserve']['decimals']))
        
    elif tx_type == ['liquidation']:
        
        tx_row['principal_'+tx['principalReserve']['symbol']] = - int(tx['principalAmount'])*(10**(-tx['principalReserve']['decimals']))
        
        tx_row['collateral_'+tx['collateralReserve']['symbol']] = - int(tx['collateralReserve'])*(10**(-tx['collateralReserve']['decimals']))
    
    return tx_row
        

# Builiding the transaction DataFrame

Now we use the `format_tx` function on all deposits, redeems, borrows, repays and liquidations, through each of their historic arrays obtained in the `.json` format file.


After this, all of the formatted transactions are combined into a single list that will later become a dataframe with each row being the change in the user's position with the transaction.

In [5]:
df_list = []

for user in user_data:
    user_list = []
    
    for tx in user['depositHistory']:
        tx['tx_type'] = 'deposit'
        user_list.append(tx)
        
    for tx in user['redeemUnderlyingHistory']:
        tx['tx_type'] = 'redeem'
        user_list.append(tx)
        
    for tx in user['borrowHistory']:
        tx['tx_type'] = 'borrow'
        user_list.append(tx)
    
    for tx in user['repayHistory']:
        tx['tx_type'] = 'repay'
        user_list.append(tx)
        
    for tx in user['liquidationCallHistory']:
        tx['tx_type'] = 'liquidation'
        user_list.append(tx)
        
    user_list.sort(key = lambda tx : tx['timestamp'] )
    
    for tx in user_list:
        df_row = {}
        df_list.append(format_tx(tx,user))

In [6]:
df_list

[{'tx_id': '15306019:258:0xc305a55fa366754bb8f5cbff86fc23eb7c7cb0289e51d92b64cd9391946d5780:721:721',
  'tx_type': 'deposit',
  'timestamp': 1660021146,
  'account': '0x000000000000512644392395185ad535410f98c2',
  'collateral_WETH': 1750.0000000000002,
  'principal_WETH': 0.0},
 {'tx_id': '15312751:89:0x6108df238ef0c8ef508de06628dab9eca646b2eeff77bbeb0a870f930eecf53e:549:549',
  'tx_type': 'deposit',
  'timestamp': 1660112664,
  'account': '0x000000000000512644392395185ad535410f98c2',
  'collateral_WETH': 7700.000000000001,
  'principal_WETH': 0.0},
 {'tx_id': '15363845:0:0x22e1e3edf1d8e44a6146a57bcf73dc78b7ea138cbfd838024bc0628947b08e48:16:16',
  'tx_type': 'redeem',
  'timestamp': 1660809650,
  'account': '0x000000000000512644392395185ad535410f98c2',
  'collateral_WETH': -9454.774906815417,
  'principal_WETH': 0.0},
 {'tx_id': '15312751:89:0x6108df238ef0c8ef508de06628dab9eca646b2eeff77bbeb0a870f930eecf53e:557:557',
  'tx_type': 'borrow',
  'timestamp': 1660112664,
  'account': '0x000

We create a `DataFrame` using the transaction data list obtained before.


Every transaction affects only one reserve, so when the `DataFrame` is created we get a lot of `NaN` values in each row for reserves not affected by the row's transaction.


For this, we fill the `NaN` values with `0`s. Also, the DataFrame is ordered by `account` and `timestamp` so that transactions from a account are together and ordered chronologically.

In [7]:
df = pd.DataFrame(df_list)
df = df.fillna(0)
df = df.sort_values(['account','timestamp'])

We now group transactions by account and perform cumulative sum through `cumsum` so that the "changes" described by each transaction are aggregated, obtaining the positions of the user.

In [8]:
df.iloc[:,4:] = df.iloc[:,3:].groupby('account').cumsum()

In [10]:
df.to_csv('aave_transaction_accumulated_data.csv', index=False)

The resulting data frame has a row for each transaction done by an user and the following columns:
- **tx_id**: The Graph transaction id.


- **tx_type**: Type of transaction, either a deposit, redeem, borrow, repay or liquidation.


- **timestamp**: Timestamp at the time the user transaction was submitted.


- **account**: Account whose position is changed by the transaction.


- **collateral_\<reserve\>** : Position collateral for \<reserve\> token after the transaction.  


- **principal_\<reserve\>** : Position loan amount for \<reserve\> token after the transaction.

In [11]:
df

tx_id  tx_type  \
0       15306019:258:0xc305a55fa366754bb8f5cbff86fc23e...  deposit   
1       15312751:89:0x6108df238ef0c8ef508de06628dab9ec...  deposit   
3       15312751:89:0x6108df238ef0c8ef508de06628dab9ec...   borrow   
4       15319170:90:0x0ee78a520958780577a64993f01050cc...   borrow   
5       15319170:90:0x0ee78a520958780577a64993f01050cc...    repay   
...                                                   ...      ...   
798181  15487603:110:0xe92a2c961702ecea39df978de2769df...    repay   
798182  15510953:184:0xf076e78c0acfeb0fa6b118585c69a46...    repay   
798183  15510971:14:0x51355e44492d51622805dbbd87921a38...    repay   
798184  15248914:60:0x60a66f8773e8b4edeb0a03348c19e68a...  deposit   
798185  15370776:139:0xdc173d2c834ad09ca3c3c2c3d0143e5...   redeem   

         timestamp                                     account  \
0       1660021146  0x000000000000512644392395185ad535410f98c2   
1       1660112664  0x000000000000512644392395185ad535410f98c2   
3       1660112664  0x000000000000512644392395185ad535410f98c2   
4       1660199679  0x000000000000512644392395185ad535410f98c2   
5       1660199679  0x000000000000512644392395185ad535410f98c2   
...            ...                                         ...   
798181  1662516025  0xffff8941130157a0153fb5be2618b257f28d3b55   
798182  1662843446  0xffff8941130157a0153fb5be2618b257f28d3b55   
798183  1662843657  0xffff8941130157a0153fb5be2618b257f28d3b55   
798184  1659252998  0xffffd1eb5a15ceb74d81f5c6097353a1056039c7   
798185  1660905394  0xffffd1eb5a15ceb74d81f5c6097353a1056039c7   

        collateral_WETH  principal_WETH  collateral_DAI  principal_DAI  \
0             1750.0000             0.0             0.0   0.000000e+00   
1             9450.0000             0.0             0.0   0.000000e+00   
3             9450.0000             0.0             0.0   5.288292e+06   
4             9450.0000             0.0             0.0   5.288292e+06   
5             9450.0000             0.0             0.0  -2.555520e+02   
...                 ...             ...             ...            ...   
798181          36.3635             0.0             0.0   7.133044e+03   
798182          36.3635             0.0             0.0   4.533947e+03   
798183          36.3635             0.0             0.0   1.327310e+03   
798184           0.0000             0.0             0.0   0.000000e+00   
798185           0.0000             0.0             0.0   0.000000e+00   

        collateral_USDC  principal_USDC  ...  collateral_AmmUniAAVEWETH  \
0                   0.0    0.000000e+00  ...                        0.0   
1                   0.0    0.000000e+00  ...                        0.0   
3                   0.0    0.000000e+00  ...                        0.0   
4                   0.0    5.288547e+06  ...                        0.0   
5                   0.0    5.288547e+06  ...                        0.0   
...                 ...             ...  ...                        ...   
798181              0.0   -5.237840e+01  ...                        0.0   
798182              0.0   -5.237840e+01  ...                        0.0   
798183              0.0   -5.237840e+01  ...                        0.0   
798184              0.0    0.000000e+00  ...                        0.0   
798185              0.0    0.000000e+00  ...                        0.0   

        principal_AmmUniAAVEWETH  collateral_AmmUniWBTCUSDC  \
0                            0.0                        0.0   
1                            0.0                        0.0   
3                            0.0                        0.0   
4                            0.0                        0.0   
5                            0.0                        0.0   
...                          ...                        ...   
798181                       0.0                        0.0   
798182                       0.0                        0.0   
798183                       0.0                        0.0  

# Load Reserve Data

In [12]:
with open("aave_reserve_configuration_data_complete.json") as file:
    response = json.load(file)

In [13]:
reserve_data = response

In [14]:
reserve_df_list = []
reserve_history_df_lists = {}

for reserve in reserve_data:
    reserve_df_list.append({
        'id':reserve['id'],
        'symbol': reserve['symbol'],
        'decimals': reserve['decimals'],
        'lastUpdateTimestamp':reserve['lastUpdateTimestamp'],
        'baseLTVasCollateral':int(reserve['baseLTVasCollateral'])*(10**(-4)),
        'reserveLiquidationThreshold':int(reserve['reserveLiquidationThreshold'])*(10**(-4))
    })
    for config in reserve['configurationHistory']:
        config['baseLTVasCollateral'] = int(config['baseLTVasCollateral'])*(10**(-4))
        config['reserveLiquidationThreshold'] = int(config['reserveLiquidationThreshold'])*(10**(-4))
        
    reserve_history_df_lists[reserve['symbol']] = reserve['configurationHistory']

In [15]:
reserve_df = pd.DataFrame(reserve_df_list)

In [16]:
reserve_df

id           symbol  \
0   0x0000000000085d4780b73119b644ae5ecd22b3760xb5...             TUSD   
1   0x004375dff511095cc5a197a54140a24efef3a4160xac...   AmmUniWBTCUSDC   
2   0x03ab458634910aad20ef5f1c8ee96f1d6ac549190xb5...              RAI   
3   0x056fd409e1d7a124bd7017459dfea2f387b6d5cd0xb5...             GUSD   
4   0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e0xb5...              YFI   
5   0x0d8775f648430679a709e98d2b0cb6250d2887ef0xb5...              BAT   
6   0x0f5d2fb29fb7d3cfee444a200298f468908cc9420xb5...             MANA   
7   0x111111111117dc0aa78b770fa6a738034120c3020xb5...            1INCH   
8   0x1494ca1f11d487c2bbe4543e90080aeba4ba3c2b0xb5...              DPI   
9   0x1eff8af5d577060ba4ac8a29a13525bb0ee2a3d50xac...   AmmBptWBTCWETH   
10  0x1f9840a85d5af5bf1d1762f925bdaddc4201f9840xb5...              UNI   
11  0x2260fac5e5542a773aa44fbcfedf7c193bc2c5990xac...          AmmWBTC   
12  0x2260fac5e5542a773aa44fbcfedf7c193bc2c5990xb5...             WBTC   
13  0x2fdbadf3c4d5a8666bc06645b8358ab803996e280xac...    AmmUniYFIWETH   
14  0x3da1313ae46132a397d90d95b1424a9a7e3e0fce0xac...    AmmUniCRVWETH   
15  0x408e41876cccdc0f92210600ef50372656052a380xb5...              REN   
16  0x43ae24960e5534731fc831386c07755a2dc33d470xac...    AmmUniSNXWETH   
17  0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b0xb5...              CVX   
18  0x4fabb145d64652a948d72533023f6e7a623c7c530xb5...             BUSD   
19  0x50379f632ca68d36e50cfbc8f78fe16bd1499d1e0xac...   AmmGUniDAIUSDC   
20  0x514910771af9ca656af840dff83e8264ecf986ca0xb5...             LINK   
21  0x57ab1ec28d129707052df4df418d58a2d46d5f510xb5...             SUSD   
22  0x59a19d8c652fa0284f44113d0ff9aba70bd46fb40xac...    AmmBptBALWETH   
23  0x5f98805a4e8be255a32880fdec7f6728c6568ba00xb5...             LUSD   
24  0x6b175474e89094c44da98b954eedeac495271d0f0xac...           AmmDAI   
25  0x6b175474e89094c44da98b954eedeac495271d0f0xb5...              DAI   
26  0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae90xb5...             AAVE   
27  0x853d955acef822db058eb8505911ed77f175b99e0xb5...             FRAX   
28  0x8798249c2e607446efb7ad49ec89dd1865ff42720xb5...           XSUSHI   
29  0x8bd1661da98ebdd3bd080f0be4e6d9be8ce9858c0xac...    AmmUniRENWETH   
30  0x8e870d67f660d95d5be530380d0ec0bd388289e10xb5...              PAX   
31  0x956f47f50a910163d8bf957cf5846d573e7f87ca0xb5...              FEI   
32  0x9f8f72aa9304c8b593d555f12ef6589cc3a579a20xb5...              MKR   
33  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb480xac...          AmmUSDC   
34  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb480xb5...             USDC   
35  0xa2107fa5b38d9bbd2c461d6edf11b11a50f6b9740xac...   AmmUniLINKWETH   
36  0xa478c2975ab1ea89e8196811f51a7b7ade33eb110xac...    AmmUniDAIWETH   
37  0xa693b19d2931d498c5b318df961919bb4aee87a50xb5...              UST   
38  0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d50xac...    AmmUniDAIUSDC   
39  0xae7ab96520de3a18e5e111b5eaab095312d7fe840xb5...            STETH   
40  0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc0xac...   AmmUniUSDCWETH   
41  0xb6909b960dbbe7392d405429eb2b3649752b48380xac...    AmmUniBATWETH   
42  0xba100000625a3754423978a60c9317c58a424e3d0xb5...              BAL   
43  0xbb2b8038a1640196fbe3e38816f3e67cba72d9400xac...   AmmUniWBTCWETH   
44  0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f0xb5...              SNX   
45  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc20xac...          AmmWETH   
46  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc20xb5...             WETH   
47  0xc18360217d8f7ab5e7c516566761ea12ce7f9d720xb5...              ENS   
48  0xc2adda861f89bbb333c90c492cb837741916a2250xac...    AmmUniMKRWETH   
49  0xd2eec91055f07fe24c9ccb25828ecfefd4be0c410xac...  AmmGUniUSDCUSDT   
50  0xd3d2e2692501a5c9ca623199d38826e513033a170xac...    AmmUniUNIWETH   
51  0xd46ba6d942050d489dbd938a2c909a5d5039a1610xb5...             AMPL   
52  0xd5147bc8e386d91cc5dbe72099dac6c9b99276f50xb5...           RENFIL   
53  0xd533a949740bb3306d119cc777fa900ba034cd520xb5...  

In [17]:
reserve_history_dfs = {}

for reserve in reserve_history_df_lists.keys():
    reserve_history_dfs[reserve] = pd.DataFrame(reserve_history_df_lists[reserve])
    reserve_history_dfs[reserve].sort_values('timestamp')

In [18]:
cols = ['timestamp','baseLTVasCollateral','reserveLiquidationThreshold']
df = df.sort_values('timestamp')


for reserve in reserve_history_df_lists.keys():
    print(reserve)
    df = pd.merge_asof(df,reserve_history_dfs[reserve][cols],on='timestamp',direction='forward')
    df = df.rename(columns={
        'baseLTVasCollateral': 'LTV_'+reserve,
        'reserveLiquidationThreshold': 'liquidationThreshold_'+reserve
    })

TUSD
AmmUniWBTCUSDC
RAI
GUSD
YFI
BAT
MANA
1INCH
DPI
AmmBptWBTCWETH
UNI
AmmWBTC
WBTC
AmmUniYFIWETH
AmmUniCRVWETH
REN
AmmUniSNXWETH
CVX
BUSD
AmmGUniDAIUSDC
LINK
SUSD
AmmBptBALWETH
LUSD
AmmDAI
DAI
AAVE
FRAX
XSUSHI
AmmUniRENWETH
PAX
FEI
MKR
AmmUSDC
USDC
AmmUniLINKWETH
AmmUniDAIWETH
UST
AmmUniDAIUSDC
STETH
AmmUniUSDCWETH
AmmUniBATWETH
BAL
AmmUniWBTCWETH
SNX
AmmWETH
WETH
ENS
AmmUniMKRWETH
AmmGUniUSDCUSDT
AmmUniUNIWETH
AMPL
RENFIL
CRV
AmmUSDT
USDT
KNC
AmmUniAAVEWETH
ZRX
ENJ


In [19]:
df

tx_id  tx_type  \
0       11363052:76:0x3081a4ac6666e748fee8bc216b86c47c...  deposit   
1       11363059:168:0x6d18b83407c28345c8bb09f458b36cb...  deposit   
2       11367463:173:0x7f10004def891a9965e972e57ecb8b9...  deposit   
3       11367499:87:0xdd1d259637c499ff61b0d142e7bb6639...  deposit   
4       11367752:130:0x870bf17a6373c10290d626134936230...  deposit   
...                                                   ...      ...   
798181  15511986:370:0xeb5da75f2ed58a8c52a0fffe202cc27...   redeem   
798182  15511987:86:0xe4b977f6128903ac6362ad2b5671cd72...    repay   
798183  15511987:146:0xe419bf298f435ed19ae176be3fb84dd...   borrow   
798184  15511987:86:0xe4b977f6128903ac6362ad2b5671cd72...  deposit   
798185  15511992:127:0x077ffc5456cbe59154be645af8bff69...   borrow   

         timestamp                                     account  \
0       1606777900  0xc4a936b003bc223df757b35ee52f6da66b062935   
1       1606778006  0xc4a936b003bc223df757b35ee52f6da66b062935   
2       1606836555  0xdad4c11e8cc6a5c37808d3b31b3b284809f702d1   
3       1606836949  0xdad4c11e8cc6a5c37808d3b31b3b284809f702d1   
4       1606840248  0xbd723fc4f1d737dcfc48a07fe7336766d34cad5f   
...            ...                                         ...   
798181  1662858582  0x759df8896f312ff32a749ec0ea17947326d55e9d   
798182  1662858644  0x2553be80118524f65649a7dae6c4d8db2aca04db   
798183  1662858644  0xcefcca169357a18eb8c3f230f92b58562e48cae5   
798184  1662858644  0x2553be80118524f65649a7dae6c4d8db2aca04db   
798185  1662858720  0x5827bc46897206a5de0eab599fd289798584dd79   

        collateral_WETH  principal_WETH  collateral_DAI  principal_DAI  \
0              0.000100        0.000000        0.000000            0.0   
1              0.000200        0.000000        0.000000            0.0   
2              0.000000        0.000000        0.000000            0.0   
3              0.001000        0.000000        0.000000            0.0   
4              0.001000        0.000000        0.000000            0.0   
...                 ...             ...             ...            ...   
798181         0.100000        0.000000        0.000000            0.0   
798182         1.791411        0.000000        0.000000            0.0   
798183         0.000000       -0.023322   222258.898419            0.0   
798184         1.791411        0.000000        0.000000            0.0   
798185       152.705551        0.000000        0.000000            0.0   

        collateral_USDC  principal_USDC  ...  LTV_USDT  \
0              0.000000        0.000000  ...       0.0   
1              0.000000        0.000000  ...       0.0   
2              0.000000        0.000000  ...       0.0   
3              0.000000        0.000000  ...       0.0   
4              0.000000        0.000000  ...       0.0   
...                 ...             ...  ...       ...   
798181         0.000000        0.000000  ...       0.0   
798182         0.000000       -0.204782  ...       0.0   
798183    789177.409278        0.000000  ...       0.0   
798184         0.000000       -0.204782  ...       0.0   
798185    492460.190187        0.000000  ...       0.0   

        liquidationThreshold_USDT  LTV_KNC  liquidationThreshold_KNC  \
0                             0.0    0.855                      0.86   
1                             0.0    0.855                      0.86   
2                             0.0    0.855                      0.86   
3                             0.0    0.855                      0.86   
4                             0.0    0.855                      0.86   
...                           ...      ...                       ...   
798181                        0.0    0.600                      0.70   
798182                        0.0    0.600                      0.70   
798183                        0.0    0.600                      0.70   
798184                        0.0    0.600                      0.70   
798185                        0.0    0.6